## Summary

Calculate features using [AlphaFold](https://github.com/deepmind/alphafold) (for the wildtype protein only).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_alphafold_wt.ipynb)"
export DATASET_NAME="humsavar"
export DATASET_PATH="30_humsavar/humsavar-gby-protein-waln.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=12557

sbatch --export NOTEBOOK_PATH,DATASET_NAME,DATASET_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-9999 --time 3:00:00 --gres=gpu:p100:1 --mem=18G ../scripts/run_notebook_gpu.sh

export ARRAY_TASK_OFFSET=10000
sbatch --export NOTEBOOK_PATH,DATASET_NAME,DATASET_PATH,ORIGINAL_ARRAY_TASK_COUNT,ARRAY_TASK_OFFSET --array=0-1000 --time 3:00:00 --gres=gpu:v100l:1 --mem=32G ../scripts/run_notebook_gpu.sh

export ARRAY_TASK_OFFSET=10000
sbatch --export NOTEBOOK_PATH,DATASET_NAME,DATASET_PATH,ORIGINAL_ARRAY_TASK_COUNT,ARRAY_TASK_OFFSET --array=1000-2557 --time 24:00:00 --gres=gpu:v100l:1 --mem=46G ../scripts/run_notebook_gpu.sh
```

---

## Imports

In [ ]:
%env TF_FORCE_UNIFIED_MEMORY=1
%env XLA_PYTHON_CLIENT_MEM_FRACTION=4.0

In [ ]:
import concurrent.futures
import os
import re
import subprocess
import sys
import tempfile
from pathlib import Path

import jax
import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.alphafold import AlphaFold, AlphaFoldAnalyzeError, AlphaFoldBuildError
from jax.lib import xla_bridge
from kmbio import PDB
from tqdm.notebook import tqdm

## Parameters

In [ ]:
NOTEBOOK_DIR = Path("31_run_alphafold_wt").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

In [ ]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir

print(tempfile.gettempdir())

In [ ]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")
ARRAY_TASK_OFFSET = int(os.getenv("ARRAY_TASK_OFFSET", "0"))

TASK_ID = (int(TASK_ID) + ARRAY_TASK_OFFSET) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT, ARRAY_TASK_OFFSET

In [ ]:
DEBUG = TASK_ID is None

if DEBUG:
#     DATASET_NAME = "cagi6-sherloc"
#     DATASET_PATH = str(
#         NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
#     )
#     TASK_ID = 3500
#     TASK_COUNT = 4182
    DATASET_NAME = "humsavar"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath("30_humsavar", "humsavar-gby-protein-waln.parquet")
    )
    TASK_ID = 10000
    TASK_COUNT = 12557
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()
    assert TASK_ID is not None
    assert TASK_COUNT is not None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

In [ ]:
device = xla_bridge.get_backend().platform

device

In [ ]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

In [ ]:
if output_file.is_file():
    raise Exception("Already finished!")

## Workspace

### Initialize model

In [ ]:
AlphaFold.load_model(device=device)

### Load data

In [ ]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

In [ ]:
assert TASK_COUNT == pfile.num_row_groups

In [ ]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [ ]:
display(input_df.head(2))
print(len(input_df))

In [ ]:
protein_id_column = None

for col in ["protein_id", "uniprot_id"]:
    if col in input_df:
        protein_id_column = col
        
assert protein_id_column is not None
protein_id_column

In [ ]:
tup = next(input_df.itertuples(index=False))

iterable_fields = []
for field in tup._fields:
    if field in [protein_id_column]:
        continue
    try:
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError:
        pass

iterable_fields

In [ ]:
if DEBUG:
    for field in iterable_fields:
        input_df[field] = input_df[field].str[:3]

### Score mutations

In [ ]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None:
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False

    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False

    return True

In [ ]:
def predictions_to_embeddings(predictions):
    return {
        "plddt": pa.array([predictions["plddt"]]),
        "max_predicted_aligned_error": pa.array(
            [predictions["max_predicted_aligned_error"].item()]
        ),
        "ptm": pa.array([predictions["ptm"].item()]),
        #
        "experimentally_resolved": pa.array(
            [predictions["experimentally_resolved"]["logits"].to_py().tolist()]
        ),
        "predicted_lddt": pa.array([predictions["predicted_lddt"]["logits"].to_py().tolist()]),
        #
        "msa_first_row": pa.array(
            [predictions["representations"]["msa_first_row"].to_py().tolist()]
        ),
        "single": pa.array([predictions["representations"]["single"].to_py().tolist()]),
        "structure_module": pa.array(
            [predictions["representations"]["structure_module"].to_py().tolist()]
        ),
        # Pairwise metrics
        "distogram": pa.array([data.predictions["distogram"]["logits"].to_py().tolist()]),
        "distogram_bin_edges": pa.array(
            [data.predictions["distogram"]["bin_edges"].to_py().tolist()]
        ),
        "masked_msa": pa.array([data.predictions["masked_msa"]["logits"].to_py().tolist()]),
        "predicted_aligned_error": pa.array([data.predictions["predicted_aligned_error"].tolist()]),
        "aligned_confidence_probs": pa.array(
            [data.predictions["aligned_confidence_probs"].tolist()]
        ),
        "msa": pa.array([data.predictions["representations"]["msa"].to_py().tolist()]),
        "pair": pa.array([data.predictions["representations"]["pair"].to_py().tolist()]),
    }

In [ ]:
assert len(input_df) == 1

for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

    data = AlphaFold.build(tup.sequence, ligand_sequence=None, msa=tup.alignment)

    results = {"protein_id": pa.array([tup.protein_id])} | predictions_to_embeddings(
        data.predictions
    )

    del data

In [ ]:
# results = {}
# for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
#     assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

#     data = AlphaFold.build(tup.sequence, ligand_sequence=None, msa=tup.alignment)

#     results.setdefault("protein_id", []).append(tup.protein_id)
#     for key, value in predictions_to_embeddings(data.predictions).items():
#         results.setdefault(key, []).append(value)

#     del data

In [ ]:
table = pa.Table.from_pydict(results)
del results

### Save results

In [ ]:
if not DEBUG:
    pq.write_table(table, output_file)
else:
    pq.write_table(table, Path(tempfile.gettempdir(), output_file.name))